In [9]:
import numpy as np
import pandas as pd

#Neural Network Classifier
class NN(object):
    def __init__(self, layers, biases, weights):
            self.num_layers = len(layers)
            self.sizes = layers
            self.biases = biases
            self.weights = []
            count = 0
            for i in range(len(layers) - 1):
                l1 = self.sizes[i]
                l2 = self.sizes[i+1]
                temp_array = np.array(weights[count: count+l1*l2])
                temp_array.shape = (l2, l1)
#                 print(i, temp_array)
                self.weights.append(temp_array)
                count += l1*l2
#             print("self.weights = " + str(self.weights))

    #backpropagation algorithm
    def backprop(self, x, y):
        update_b = [np.zeros(b.shape) for b in self.biases]
        update_w = [np.zeros(w.shape) for w in self.weights]
        activation = np.array(x)

        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = (np.dot(w, activation))
            z.shape = (w.shape[0], 1)
            # print("z.shape = ", z.shape)
            z = z+b;
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        delta = self.cd(activations[-1], y)*sd(zs[-1])
        update_b[-1] = delta
        update_w[-1] = np.dot(delta, activations[-2].transpose())
        activations[0].shape = (self.sizes[0], 1)

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sd(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta)*sp
            update_b[-l] = delta
            update_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return update_b, update_w

    #accuracy calculation
    def accuracy(self, X, Y):
        y = self.fprop(X)
#         print("Forward propagation results = " + str(y))
        y = [1 if p>0.5 else 0 for p in y[0]]
        acc = sum([1 if f==F else 0 for f, F in zip(y, Y)])
        tp = 0
        fp = 0
        fn = 0
        for i, j in zip(y, Y):
            if i==1 and j==1:
                tp=tp+1
            if i==1 and j==0:
                fp=fp+1
            if i==0 and j==1:
                fn=fn+1
        precision=0
        try:
            precision = float(tp)/(tp+fp)
        except Exception as e:
            print(e)
        recall=0
        try:
            recall = float(tp)/(tp+fn)
        except Exception as e:
            print(e)
        
        return float(acc)/10.0, precision, recall

    #forward propagation
    def fprop(self, a):
        a = a.transpose()
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    #Stochastic GD : one record at a time
    def Stoch_GD(self, X, Y, alpha, i):
            for _ in range(10):
                for x,y in zip(X, Y):
                    delta_update_b, delta_update_w = self.backprop(x, y)
                    self.weights = [w-alpha*nw for w, nw in zip(self.weights, delta_update_w)]
                    self.biases = [b-alpha*nb for b, nb in zip(self.biases, delta_update_b)]

    #difference in output
    def cd(self, output, y):
        return output-y

    #return the weights and biases
    def get_weights_and_biases(self):
        return self.weights, self.biases

def getData():
    df = pd.read_csv('IRIS.csv')
    df = df.sample(frac=1)
    X = df.as_matrix()
    y = X[:, -1]
    y = [0 if x == 'Iris-setosa'  else 1 for x in y]
    X = X[:, 0:-1]
    return X, y

def sigmoid(z):
    z = z.astype(float)
    return 1.0/(1.0+np.exp(-z))


def sd(z):
    return sigmoid(z)*(1-sigmoid(z))

def selection(ranks, population):
    sorted(ranks, key = lambda ranks: ranks[0], reverse=True)
    new_population = []
    ranks = ranks[:5]
    temp_pop = []
    for i in range(len(ranks)):
        temp_pop.append(population[ranks[i][1]])
    for i in range(0, 6):
        new_population += temp_pop

    return new_population

def crossover(population, c_rate):
    l = len(population)
    num_cross_overs = (int)(l / c_rate)
    for i in range(num_cross_overs):
        l1 = (int)(np.random.rand()*l - 1)
        l2 = (int)(np.random.rand()*l - 1)
        parent1 = population[l1]
        parent2 = population[l2]
        parent1 = parent1.transpose()
        parent2 = parent2.transpose()
        parent1 = parent1[0].tolist()
        parent2 = parent2[0].tolist()
#         print("Parent 1 = " + str(parent1))
#         print("Parent 2 = " + str(parent2))
        l = len(parent1)
        l = (int)(l/2)
#         print("Parent1[:l] = " + str(parent1[:l]))
#         print("Parent2[l:] = " + str(parent2[l:]))
        child = parent1[:l] + parent2[l:]
        
#         print("Length Child = " + str(len(child)))
        child = np.array(child)
        child.shape = (25, 1)
#         print("Child = " + str(child))
        population.append(child)
        
    return population

def k_fold_algo(k, X, y):
    # print("y = ", y)
    test = []
    train = []
    test_y = []
    train_y = []
    for i in range(0, len(X)):
        if i%10==k:
            test.append(X[i])
            test_y.append(y[i])
        else:
            train.append(X[i])
            train_y.append(y[i])

    return np.array(train), np.array(test), np.array(train_y), np.array(test_y)

if __name__ == '__main__':
        X, y = getData()
#         print("y = ", y)
        layers = [4,5,1]
        tot = 0
        for i in range(len(layers) - 1):
            l1 = layers[i]
            l2 = layers[i+1]
            tot += l1*l2
        
        weights_array = []
        biases_array = []
        for i in range(0, 30):
            biases = [np.random.randn(l, 1) for l in layers[1:]]
            weights = np.random.randn(tot, 1)
            weights_array.append(weights)
            biases_array.append(biases)
        
        for l in range(0,100):
            print("l = " + str(l))
            rank_list = []
            accuracy_sum = 0
            for i in range(0, 30):
                print("i = " + str(i))
    #             print("Iteration #" + str(i))
                biases = biases_array[i]
                weights = np.array(weights_array[i], copy=True)
    #             print("Weights array = " + str(weights))
                net  = NN([4,5, 1], biases, weights)
                # print "Initial Random Bias:"+str(net.biases)
                # print "Initial Random Weights:"+str(net.weights)
                X_train, X_test, y_train, y_test = k_fold_algo(0, X, y)
    #             for k in range(2):
                net.Stoch_GD(X_train, y_train, 0.1, i)
                answers = net.accuracy(X_test, y_test)
    #             print("answers = " + str(answers))
                accuracy_sum = accuracy_sum+answers[0]
                rank_list.append((answers[0], i))
                weights, biases = net.get_weights_and_biases()
                temp = []
                for j in range(len(layers)-1):
                    weights1 = np.array(weights[j], copy=True)
                    weights1 = weights1.reshape((layers[j+1]*layers[j], 1))
                    temp.append(weights1)
                temp = np.vstack((temp[0], temp[1]))
    #             print("temp = " + str(temp))
                weights_array[i] = temp
                biases_array[i] = biases
    #             print("Accuracy "+str(i+1)+" : "+str(answers[0]))
    #             print("Precision is: "+str(answers[1]))
    #             print("Recall is:"+str(answers[2]))
    #         for i in range(len(weights_array)):
    #         temp = weights_array[0]
    #         print(temp.transpose())
    #         temp = np.array(temp)
    #         temp = temp.reshape((1, tot))
    #         print("Demo = " + str(temp))
            new_pop = selection(rank_list, weights_array)
            new_pop = crossover(weights_array, 0.25)
            new_pop = new_pop[-30:]
            weights_array = new_pop
#             print("New Population = " + str(new_pop))
    #             print("Len New Pop = " + str(len(new_pop)))
            print("Avg Accuracy for iteration #{}: {}".format(l, accuracy_sum/30.0))


l = 0
i = 0
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 20
i = 21
i = 22
i = 23
i = 24
i = 25
i = 26
i = 27
i = 28
i = 29
Avg Accuracy for iteration #0: 0.9533333333333334
l = 1
i = 0
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 20
i = 21
i = 22
i = 23
i = 24
i = 25
i = 26
i = 27
i = 28
i = 29
Avg Accuracy for iteration #1: 0.9766666666666667
l = 2
i = 0
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 20
i = 21
i = 22
i = 23
i = 24
i = 25
i = 26
i = 27
i = 28
i = 29
Avg Accuracy for iteration #2: 1.0
l = 3
i = 0
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 20
i = 21
i = 22
i = 23
i = 24
i = 25
i = 26
i = 27
i = 28
i = 29
Avg Accuracy for iteration #3: 0.95333333

KeyboardInterrupt: 